In [1]:
!python --version

Python 3.10.14


In [2]:
!pip install -r requirements.txt

In [3]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_community.vectorstores import Chroma
import google.generativeai as genai
from langchain.text_splitter import CharacterTextSplitter


In [4]:
# with open('.env', 'w') as f:
#     f.write('GOOGLE_API_KEY=\n')

In [5]:
load_dotenv('.env')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [6]:
llm = ChatGoogleGenerativeAI(model="models/gemini-pro")

In [7]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
pdf_directory = "data/"
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

text_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=2500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

all_pages = []

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split(text_splitter)
    all_pages.extend(pages)

In [ ]:
db = Chroma.from_documents(pages, embedding)
db

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 7})

In [ ]:
template = """
Вы - помощник с искусственным интеллектом.
Отвечайте, исходя из предоставленного контекста.

context: {context}
input: {input}
answer:
"""

In [ ]:
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
query = "Каким должен быть угол отбортовки?"
response = retrieval_chain.invoke({"input": query})

In [ ]:
print(response["answer"])